# arUSD trading volume by wallet
It is assumed that the address swapping is is also swapping out. Synth-to-synth swaps and transfers of synths between wallets are not considered.

arUSD contract: 0x025AB35fF6AbccA56d57475249baaEae08419039
Swapping contract: 0x948011e8ca8df1e9c83fee88967a5fc30c7a604b
Fee holder contract: 0xc768915863a333db9bb871bd687dd0a0ae41a3be

In [1]:
import os
from pathlib import Path
#import scripts.utils as utils
import utils
import pandas as pd
#from IPython.display import display, HTML
import matplotlib.pyplot as plt
#%matplotlib inline
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

start_date = "2022-06-01"


query_variables = {"start_date": start_date}

query = """
query ($start_date: ISO8601DateTime) {
ethereum(network: avalanche) {
    transfers(
			options: {
				limit:100000,
				desc: ["amount"]
			}
			date: {since: $start_date}
			receiver:{notIn:["0xc768915863a333db9bb871bd687dd0a0ae41a3be","0x948011e8ca8df1e9c83fee88967a5fc30c7a604b"]}
			sender:{is:"0x0000000000000000000000000000000000000000"}
			currency: {is: "0x025AB35fF6AbccA56d57475249baaEae08419039"}
		) {
		receiver {address}
		amount
		currency {symbol, address}
   }
  }
}
"""

swap_out_result = utils.bitqueryAPICall(query, query_variables)

swap_out = []
for r in swap_out_result["data"]["ethereum"]["transfers"]:
    swap_out.append([r["receiver"]["address"], r["amount"]])

query = """
query ($start_date: ISO8601DateTime) {
ethereum(network: avalanche) {
    transfers(
			options: {
				desc: ["amount"]
			} 
			date: {since: $start_date}
			receiver:{is:"0x948011e8ca8df1e9c83fee88967a5fc30c7a604b"}
			sender:{notIn:["0x0000000000000000000000000000000000000000", "0xc768915863a333db9bb871bd687dd0a0ae41a3be"]}
			currency: {is: "0x025AB35fF6AbccA56d57475249baaEae08419039"}
		) {
		sender {address}
		amount
		currency {symbol, address}
   }
  }
}
"""

swap_in_result = utils.bitqueryAPICall(query, query_variables)

swap_in = []
for r in swap_in_result["data"]["ethereum"]["transfers"]:
    swap_in.append([r["sender"]["address"], r["amount"]])

df_out = utils.df(swap_out, columns=["address", "amount_out"])
df_in = utils.df(swap_in, columns=["address", "amount_in"])

int_df = pd.merge(df_out, df_in, how='inner', on=['address'])

# sum of amount_in and amount_out
int_df['amount'] = int_df['amount_in'] + int_df['amount_out']

int_df


# display(int_df)


,address,amount_out,amount_in,amount
0,0x02aceac337c6039eb9b0d2e43b730241ca620f73,1.255269e+06,1.241480e+06,2.496749e+06
1,0x42326781b4776719ec87d57246d19c79ea1f7629,6.999413e+05,6.925951e+05,1.392536e+06
2,0xd939c534bbb89463b1bef66d9201f4b287352e6d,2.585849e+05,2.496011e+05,5.081860e+05
3,0xc5e297e508364c9b247a4b6209c5da73d9880867,8.648328e+04,8.338842e+04,1.698717e+05
4,0x16a522067b5d88a803dd43f64dea23d28a5d3cc9,3.578873e+04,7.350000e+02,3.652373e+04
5,0xb7113ce37c448a4ebf5f94c1e360f3dd52d35729,3.148540e+04,3.625572e+03,3.511098e+04
6,0xd35f615565b16b46f09c3e96553d175483978b73,3.139366e+04,2.983940e+04,6.123306e+04
7,0x531563f28b6d595583a9158475d1048372a03e2c,3.084436e+04,2.797649e+04,5.882086e+04
8,0x364705913b668b17f7595fba78b4e8888f5927f2,2.624599e+04,7.372211e+03,3.361820e+04
9,0x274aaa262e11cac18865a3e8d0cd43675d0a4fe1,2.589218e+04,2.219444e+04,4.808662e+04
